In [41]:
from optibook.synchronous_client import Exchange
import time
import matplotlib.pyplot as plt
import numpy as np
import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


In [87]:
e = Exchange()
a = e.connect()

2021-01-23 22:12:03,848 [asyncio   ] [MainThread  ] Using selector: EpollSelector


In [4]:
philips_a = 'PHILIPS_A'
philips_b = 'PHILIPS_B'

In [5]:
def my_position():
    # Returns all current positions with cash invested
    positions = e.get_positions_and_cash()
    for p in positions:
        print(p, positions[p])
        


In [20]:
my_position()

PHILIPS_A {'volume': -400, 'cash': 32199.099999999988}
PHILIPS_B {'volume': 0, 'cash': 4749.600000000029}


In [7]:
# Delete all outstanding orders
def delete_outstanding(instrument):
    outstanding = e.get_outstanding_orders(instrument)
    for o in outstanding.values():
        result = e.delete_order(instrument, order_id=o.order_id)
        print(f"Deleted order id {o.order_id}: {result}")

In [8]:
# See all your outstanding orders
def view_outstanding(instrument):
    outstanding = e.get_outstanding_orders(instrument)
    for o in outstanding.values():
        print(f"Outstanding order: order_id({o.order_id}), instrument_id({o.instrument_id}), price({o.price}), volume({o.volume}), side({o.side})")

In [9]:
def best_selling_price(instrument):
    book = e.get_last_price_book(instrument)
    best = book.bids[0].price
    return best

def best_buying_price(instrument):
    book = e.get_last_price_book(instrument)
    best = book.asks[0].price
    return best
    
    

In [340]:
delete_outstanding(philips_a)

Deleted order id 261036: True


In [11]:
view_outstanding(philips_a)
view_outstanding(philips_b)

In [12]:
def average_a(timesteps):
    # Returns all public tradeticks since the instantiation of the Exchange
    tradeticks = e.get_trade_tick_history(philips_a)
    prices = []
    for t in tradeticks[-timesteps:]:
        if t.price<100:
            prices.append(t.price)

    return np.mean(prices)

def average_b(timesteps):
    # Returns all public tradeticks since the instantiation of the Exchange
    tradeticks = e.get_trade_tick_history(philips_b)
    prices = []
    for t in tradeticks[-timesteps:]:
        if t.price<100:
            prices.append(t.price)

    return np.mean(prices)

def get_range_b(timesteps):
    tradeticks = e.get_trade_tick_history(philips_b)
    prices = []
    for t in tradeticks[-timesteps:]:
        if t.price<100:
            prices.append(t.price)
    return [np.percentile(prices, 99), np.mean(prices), np.percentile(prices, 1)]
    
def get_range_a(timesteps):
    tradeticks = e.get_trade_tick_history(philips_a)
    prices = []
    for t in tradeticks[-timesteps:]:
        if t.price<100:
            prices.append(t.price)
    return [np.percentile(prices, 99), np.mean(prices), np.percentile(prices, 1)]
    


In [13]:
average_a(6000)
print(get_range_b(10000))

[93.9, 93.22413793103448, 92.5]


In [100]:
my_position()
print(np.mean(bought_prices))
print(best_selling_price(philips_a))
my_position()


PHILIPS_A {'volume': -73, 'cash': -991671.4999999969}
PHILIPS_B {'volume': 0, 'cash': -5406.900000000038}
95.35357142857144
91.5
PHILIPS_A {'volume': -73, 'cash': -991671.4999999969}
PHILIPS_B {'volume': 0, 'cash': -5406.900000000038}


In [99]:
current_position = e.get_positions_and_cash()
volume_left = current_position[philips_a]['volume']
print(volume_left)
best_sell = best_selling_price(philips_a)
e.insert_order(philips_a, price=best_sell, volume=100, order_type='limit', side='ask')

27


427053

In [46]:
bought_prices = [average_b(200)+0.5]
weighted_prices_a = [average_a(200)]

weighted_prices_b = [average_b(200)]

In [ ]:
start_pnl = e.get_pnl()

timer = 0

while timer<2000:
    stats_a = get_range_a(1000)
    stats_b = get_range_b(1000)
    
    max_range_a = stats_a[0]-stats_a[1]
    low_range_b = stats_b[1]-stats_b[2]
    
    book_a = e.get_last_price_book(philips_a)
    book_b = e.get_last_price_book(philips_b)
    
    if len(book_a.asks)!=0 and len(book_a.bids)!=0:
        current_ask_a = book_a.asks[0].price
        current_bid_a = book_a.bids[0].price
    if len(book_b.asks)!=0 and len(book_b.bids)!=0:
        current_ask_b = book_b.asks[0].price
        current_bid_b = book_b.bids[0].price
    #print(current_ask)
    
    average = average_b(2000)
    #print(average)
    positions = e.get_positions_and_cash()
    volume_left_a = positions[philips_a]['volume']
    volume_left_b = positions[philips_b]['volume']
    
    # SELLING A WHEN HIGH
    if current_bid_a>np.mean(weighted_prices_a) and volume_left_a>-400:
        price = current_bid_a
        volume = max(int(10*(current_bid_a-stats_a[1])/max_range_a),1)
        buying_order = e.insert_order(philips_a, price=current_bid_a, volume=volume, side = 'ask', order_type = 'ioc')
        print('Selling %s A at price %s ****** Expect to buy at %s'%(volume, current_bid_a, np.mean(weighted_prices_a)))
        bought_prices.append(price)
        updated_weights = [price]*volume
        weighted_prices_a = updated_weights + weighted_prices_a
        time.sleep(1)
        timer+=1
        
    
    # BUYING A WHEN LOW
    if current_ask_a<np.mean(weighted_prices_a) and volume_left_a<50:
        sell_volume = min(int(10*(np.mean(weighted_prices_a)-current_ask_a)*max_range_a), -volume_left_a)
        if sell_volume<1:
            sell_volume = 1
        selling_order = e.insert_order(philips_a, price=current_ask_a, volume=sell_volume, side='bid', order_type='limit')  
        print('Buying  %s A at price %s'%(sell_volume, str(current_ask_a)))
        weighted_prices_a = weighted_prices_a[sell_volume:]
        time.sleep(0.01)
        timer +=1
        
    # BUYING B THEY LOW
    if current_ask_b<np.mean(weighted_prices_b) and volume_left_b<400:
        price = current_ask_b
        volume = max(int(10*((stats_b[1]-current_ask_b)/low_range_b)),1)
        buying_order = e.insert_order(philips_b, price=current_ask_b, volume=volume, side = 'bid', order_type = 'ioc')
        print('Buying %s B at price %s ****** Expect to sell at %s'%(volume, current_ask_b, np.mean(weighted_prices_b)))
        bought_prices.append(price)
        updated_weights = [price]*volume
        weighted_prices_b = updated_weights + weighted_prices_b
        time.sleep(1)
        timer+=1
        
    
    # SELLING B WHEN HIGH
    if current_bid_b>np.mean(weighted_prices_b) and volume_left_b>0:
        sell_volume = min(int(10*(current_bid_b-np.mean(weighted_prices_b)*low_range_b)), volume_left_b)
        if sell_volume<1:
            sell_volume = 1
        selling_order = e.insert_order(philips_b, price=current_bid_b, volume=sell_volume, side='ask', order_type='limit')  
        print('Selling %s B at price %s'%(sell_volume, str(current_bid_b)))
        weighted_prices_b = weighted_prices_b[sell_volume:]
        time.sleep(0.01)
        timer +=1
          
    if timer%10==0:
#         weighted_prices_a.append(average_a(5)-0.2)
        weighted_prices_a = [average_a(10)-0.2] + weighted_prices_a
        
#         weighted_prices_b.append(average_b(5)+0.3)
        weighted_prices_b = [average_b(10)+0.2] + weighted_prices_b
        print('Updated Weights A=%s B=%s'%(np.mean(weighted_prices_a), np.mean(weighted_prices_b)))
    time.sleep(0.01)
    
    
end_pnl = e.get_pnl()

print('Pnl of stategy = %s'%(str(end_pnl - start_pnl)))

Buying  1 A at price 91.30000000000001
Buying 7 B at price 91.5 ****** Expect to sell at 91.74671875
Buying  1 A at price 91.30000000000001
Buying 7 B at price 91.5 ****** Expect to sell at 91.7223943661972
Buying  1 A at price 91.2
Buying 4 B at price 91.60000000000001 ****** Expect to sell at 91.70243589743589
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Updated Weights A=97.8781994047619 B=91.69650602409638
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Updated Weights A=97.85998381877022 B=91.69559523809524
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.4
Buying  1 A at price 91.5
Buy

In [92]:
start_pnl = e.get_pnl()

timer = 0

while timer<5:
    book_a = e.get_last_price_book(philips_a)
    book_b = e.get_last_price_book(philips_b)
    
    if len(book_a.asks)!=0 and len(book_a.bids)!=0:
        current_ask_a = book_a.asks[0].price
        current_bid_a = book_a.bids[0].price
    if len(book_b.asks)!=0 and len(book_b.bids)!=0:
        current_ask_b = book_b.asks[0].price
        current_bid_b = book_b.bids[0].price
        
    b_buy_price = current_ask_b
    a_buy_price = current_ask_a
    
    a_order = e.insert_order(philips_a, price=a_buy_price, volume=200, order_type='ioc', side='bid')
    b_order = e.insert_order(philips_b, price=b_buy_price, volume=200, order_type='ioc', side='bid')
    print('Entered Selling Phase')
    sold = 0
    while True:
        
        book_a = e.get_last_price_book(philips_a)
        book_b = e.get_last_price_book(philips_b)
    
        if len(book_a.asks)!=0 and len(book_a.bids)!=0:
            current_ask_a = book_a.asks[0].price
            current_bid_a = book_a.bids[0].price
        if len(book_b.asks)!=0 and len(book_b.bids)!=0:
            current_ask_b = book_b.asks[0].price
            current_bid_b = book_b.bids[0].price
        positions = e.get_positions_and_cash()
        volume_left_a = positions[philips_a]['volume']
        volume_left_b = positions[philips_b]['volume']
            
        
        if current_bid_a>a_buy_price+0.2 and volume_left_a>0:
            a_order = e.insert_order(philips_a, price=current_bid_a, volume=volume_left_a, order_type='limit', side='ask')
            sold +=1
        if current_bid_b>b_buy_price+0.2 and volume_left_b>0:
            a_order = e.insert_order(philips_b, price=current_bid_b, volume=volume_left_b, order_type='limit', side='ask')
            sold += 1
            
        if sold==2:
            print('A and B were both sold')
            timer += 1
            break
            
        
        

Entered Selling Phase


KeyboardInterrupt: 